In [ ]:
import modulos as md
import numpy as np
import constantes as cons
import os
import sys
lista_filtrada = []

lista=md.lista_de_monedas()
#lista=md.filtradodemonedas ()
#lista=['SPELLUSDT', 'AXSUSDT', 'ETHUSDT', 'BNBUSDT', 'OGNUSDT', 'CYBERUSDT', '1000PEPEUSDT', 'COMPUSDT', 'ADAUSDT', 'BCHUSDT', 'RUNEUSDT', 'MATICUSDT', 'BLZUSDT', 'HBARUSDT', 'TRBUSDT', 'WLDUSDT', 'MTLUSDT', 'BTCUSDT', 'XRPUSDT', 'OPUSDT', 'PERPUSDT', 'ARBUSDT', 'STMXUSDT', 'CFXUSDT', 'SOLUSDT', 'DOGEUSDT', 'APEUSDT', 'UNFIUSDT', 'LTCUSDT', 'STORJUSDT', 'CRVUSDT']
#lista=['STMXUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        #data,porcentajeentrada = md.estrategia_santa(symbol,tp_flag = True)
        #data,porcentajeentrada = md.estrategia_trampa(symbol,tp_flag = True)
        data = md.estrategia_haz(symbol, debug=debug)
        #data,porcentajeentrada = md.estrategia_oro('XAU',tp_flag = True)
        #######################################################################
        resultado = md.backtesting(data, plot_flag = imprimo)
        #resultado = md.backtestingsanta(data, plot_flag = imprimo, debug = True)
        ########################################################################
        #if resultado['Return [%]'] < 0: # para santa3 mazmorra
        #if data.disparo.iloc[-1]!=0: # imprime las monedas que estan en posible haz martillo
        #        print(symbol)
        if resultado['Return [%]'] > 0:
            #if ((resultado['Profit Factor'] > 2 or np.isnan(resultado['Profit Factor'])) and (resultado['Return [%]']/resultado['# Trades']) >=0.33):            
                lista_filtrada.append(symbol)
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)
#md.dibuja_patrones_triangulos (data,998)    


In [1]:
import modulos as md
import numpy as np
import os
import sys
import constantes as cons
lista_filtrada = []
lista=['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'LINKUSDT', 'BNBUSDT', 'KNCUSDT', 'DOGEUSDT', 'MKRUSDT', 'CRVUSDT', 'TRBUSDT', 'SOLUSDT', 'STORJUSDT', 'BLZUSDT', 'STMXUSDT', 'OPUSDT', 'LUNA2USDT', 'APTUSDT', 'ARBUSDT', 'AMBUSDT', 'LEVERUSDT', '1000PEPEUSDT', 'WLDUSDT', 'FRONTUSDT', 'GLMRUSDT']
lista=['BTCUSDT']

if len(lista)>1:
    imprimo=False
    debug=False
else:
    imprimo= True
    debug=True
for symbol in lista:    
    try:
        data = md.estrategia_royal(symbol,debug = False, refinado = True, file_source=False)
        resultado=md.backtesting_royal(data, plot_flag=imprimo)
        if not np.isnan(data.decisional_bajista_high.iloc[-2]) or not np.isnan(data.decisional_alcista_high.iloc[-2]):
            lista_filtrada.append(symbol)
            print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {resultado['Profit Factor']} - Win Rate [%]: {resultado['Win Rate [%]']}")
    except Exception as falla:
        _, _, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print("\nError: "+str(falla)+" - line: "+str(exc_tb.tb_lineno)+" - file: "+str(fname)+"\n")
        pass
for symbol in lista_filtrada:
    md.printandlog(cons.nombrelog,symbol,pal=1)


fecha_actual_refinar: 2023-08-28 16:00:00
fecha_actual_refinar: 2023-08-28 18:00:00
fecha_actual_refinar: 2023-08-31 14:00:00
fecha_actual_refinar: 2023-09-08 07:00:00
fecha_actual_refinar: 2023-09-10 21:00:00
fecha_actual_refinar: 2023-09-11 12:00:00
fecha_actual_refinar: 2023-09-15 08:00:00
fecha_actual_refinar: 2023-09-15 10:00:00
fecha_actual_refinar: 2023-09-18 14:00:00
fecha_actual_refinar: 2023-09-18 15:00:00
fecha_actual_refinar: 2023-09-21 06:00:00
fecha_actual_refinar: 2023-09-24 20:00:00
fecha_actual_refinar: 2023-09-24 20:15:00
fecha_actual_refinar: 2023-09-24 20:30:00
fecha_actual_refinar: 2023-09-24 20:45:00
fecha elegida: 2023-09-24 20:45:00
fecha_actual_refinar: 2023-09-24 21:00:00
fecha_actual_refinar: 2023-09-24 21:15:00
fecha_actual_refinar: 2023-09-24 21:30:00
fecha_actual_refinar: 2023-09-24 21:45:00
fecha elegida: 2023-09-24 21:45:00
fecha_actual_refinar: 2023-09-27 10:00:00
fecha_actual_refinar: 2023-09-27 10:15:00
fecha_actual_refinar: 2023-09-27 10:30:00
fecha_

c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:1178: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=balance)
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a 

BTCUSDT - Return [%]: 0.0% - # Trades: 0 - Profit Factor: nan - Win Rate [%]: nan
